In [1]:
%%configure -f
{
  "conf": {
    "spark.sql.extensions": "io.delta.sql.DeltaSparkSessionExtension",
    "spark.sql.catalog.spark_catalog": "org.apache.spark.sql.delta.catalog.DeltaCatalog"
  }
}

In [2]:
from delta.tables import *
from pyspark.sql.functions import *

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1668337450421_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
deltaPath = "s3://ute-delta-lake/delta-amazon-reviews-pds/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
df_parquet = spark.read.parquet("s3://amazon-reviews-pds/parquet/product_category=Gift_Card/*.parquet")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
df_parquet = spark.read.parquet("s3://amazon-reviews-pds/parquet/product_category=Gift_Card/*.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df_parquet.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)
 |-- year: integer (nullable = true)

In [7]:
df_parquet.write.mode("overwrite").format("delta").partitionBy("year").save(deltaPath)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
df_delta = spark.read.format("delta").load(deltaPath)
df_delta.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+----+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|year|
+-----------+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+----+
|         US|   49127482|R11KUV8LZTZS9U|B0029F2570|     246986987|Amazon.com Gift C...|          5|            0|          0|   N|                Y|           gift card|the gift card was...| 2010-01-03|2010|
|         US|   25433786|R2KVFXHM91LTAX|B001H53QDK|     246986987|Amazon.com Gift C...|          5|            0|          0|   N|                Y|          Great deal

In [9]:
spark.conf.set('table.location', deltaPath)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
spark.sql("select * from df_delta").show(truncate=False)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Table or view not found: df_delta; line 1 pos 14;
'Project [*]
+- 'UnresolvedRelation [df_delta], [], false

Traceback (most recent call last):
  File "/mnt1/yarn/usercache/livy/appcache/application_1668337450421_0002/container_1668337450421_0002_01_000001/pyspark.zip/pyspark/sql/session.py", line 723, in sql
    return DataFrame(self._jsparkSession.sql(sqlQuery), self._wrapped)
  File "/mnt1/yarn/usercache/livy/appcache/application_1668337450421_0002/container_1668337450421_0002_01_000001/py4j-0.10.9.3-src.zip/py4j/java_gateway.py", line 1322, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/mnt1/yarn/usercache/livy/appcache/application_1668337450421_0002/container_1668337450421_0002_01_000001/pyspark.zip/pyspark/sql/utils.py", line 117, in deco
    raise converted from None
pyspark.sql.utils.AnalysisException: Table or view not found: df_delta; line 1 pos 14;
'Project [*]
+- 'UnresolvedRelation [df_delta], [], false




In [44]:
%%sql
SELECT * FROM  delta.`s3://ute-delta-lake/delta-amazon-reviews-pds/` LIMIT 10

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [37]:
df_delta.createOrReplaceTempView("aws_product_review")
spark.sql("select marketplace,customer_id,review_date  from aws_product_review LIMIT 30").show(30)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-----------+-----------+
|marketplace|customer_id|review_date|
+-----------+-----------+-----------+
|         US|   49127482| 2010-01-03|
|         US|   25433786| 2010-01-07|
|         US|   46729709| 2010-01-08|
|         US|   47068386| 2010-01-10|
|         US|   14833226| 2010-01-26|
|         US|   48662455| 2010-01-30|
|         US|   52951175| 2010-02-03|
|         US|   42833590| 2010-02-08|
|         US|   15279351| 2010-02-11|
|         US|   36734477| 2010-02-25|
|         US|   13534127| 2010-05-03|
|         US|   16026737| 2010-06-01|
|         US|   33420376| 2010-06-25|
|         US|   49065010| 2010-06-29|
|         US|   35662270| 2010-07-11|
|         US|   14933395| 2010-08-30|
|         US|   37053414| 2010-09-18|
|         US|   13842590| 2010-10-14|
|         US|   19553231| 2010-10-29|
|         US|   44964493| 2010-10-31|
|         US|   14118723| 2010-11-09|
|         US|   32998517| 2010-11-09|
|         US|   39853837| 2010-11-10|
|         US

In [38]:
deltaTable = DeltaTable.forPath(spark, deltaPath) #khai báo 1 datatable

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
deltaTable.update("marketplace = 'US'",{ "marketplace":"'USA'"}) # Thay đổi toàn bộ marketplace US thành USA

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
%%sql
update delta.`s3://ute-delta-lake/delta-amazon-reviews-pds/` 
set marketplace = 'USA' where marketplace = 'US' 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [45]:
df_delta.filter("verified_purchase = 'N'").show() #Xem dữ liệu trước khi xóa

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+----+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|year|
+-----------+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+----+
|        USA|   49022882|R3R9VE8FI0D0PV|BT00CTOUNS|     775486538|Amazon.com Gift C...|          5|            0|          0|   N|                N|Favorite gift of ...|I always ask for ...| 2011-01-05|2011|
|        USA|   35995131|R2QX4JAM9B375A|BT00DDC7CE|     802996579|Amazon.com Gift C...|          5|            0|          0|   N|                N|Life-saver during...

In [46]:
deltaTable.delete("verified_purchase = 'N'") #Xóa dữ liệu
df_delta.filter("verified_purchase = 'N'").show() #Show lại sau khi xóa

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-----------+---------+----------+--------------+-------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+----+
|marketplace|customer_id|review_id|product_id|product_parent|product_title|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline|review_body|review_date|year|
+-----------+-----------+---------+----------+--------------+-------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+----+
+-----------+-----------+---------+----------+--------------+-------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+----+

In [48]:
prev_version = deltaTable.history().selectExpr('max(version)').collect()[0][0] - 1
prev_version_data = spark.read.format('delta').option('versionAsOf', prev_version).load(deltaPath)
prev_version_data.filter("verified_purchase = 'N'").show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+----+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|year|
+-----------+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+----+
|        USA|   49022882|R3R9VE8FI0D0PV|BT00CTOUNS|     775486538|Amazon.com Gift C...|          5|            0|          0|   N|                N|Favorite gift of ...|I always ask for ...| 2011-01-05|2011|
|        USA|   35995131|R2QX4JAM9B375A|BT00DDC7CE|     802996579|Amazon.com Gift C...|          5|            0|          0|   N|                N|Life-saver during...

In [49]:
deltaTable.history(100).select("version", "timestamp", "operation", "operationParameters").show(truncate=False) # show ra lịch sử chỉnh sửa


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------------+---------+--------------------------------------------+
|version|timestamp          |operation|operationParameters                         |
+-------+-------------------+---------+--------------------------------------------+
|5      |2022-11-13 12:56:42|DELETE   |{predicate -> ["(verified_purchase = 'N')"]}|
|4      |2022-11-13 12:45:53|UPDATE   |{predicate -> (marketplace#2095 = US)}      |
|3      |2022-11-13 12:09:42|WRITE    |{mode -> Overwrite, partitionBy -> ["year"]}|
|2      |2022-11-13 11:10:56|WRITE    |{mode -> Overwrite, partitionBy -> ["year"]}|
|1      |2022-11-13 09:05:06|WRITE    |{mode -> Overwrite, partitionBy -> ["year"]}|
|0      |2022-11-12 16:47:24|WRITE    |{mode -> Overwrite, partitionBy -> ["year"]}|
+-------+-------------------+---------+--------------------------------------------+

In [50]:
df_time_travel = spark.read.format("delta").option("versionAsOf", 0).load(deltaPath) # đọc lại verion 0 ở lịch sử chỉnh sửa và show ra ta có thể thấy là marketplace vẫn là US trước khi sửa
df_time_travel.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+----+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|year|
+-----------+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+----+
|         US|   49127482|R11KUV8LZTZS9U|B0029F2570|     246986987|Amazon.com Gift C...|          5|            0|          0|   N|                Y|           gift card|the gift card was...| 2010-01-03|2010|
|         US|   25433786|R2KVFXHM91LTAX|B001H53QDK|     246986987|Amazon.com Gift C...|          5|            0|          0|   N|                Y|          Great deal

In [59]:
spark.sql("select * from aws_product_review where review_id in ('R315TR7JY5XODE', 'R315TR7JY5XOA1')").show() # show kết quả trước khi upsert

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------------+---------------+--------------------+-----------+----+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline|         review_body|review_date|year|
+-----------+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------------+---------------+--------------------+-----------+----+
|        USA|   38602100|R315TR7JY5XODE|B00C5UMB7S|     512447107|Amazon eGift Card...|          5|            0|          0|   N|                Y|      Gift Card|These gift cards ...| 2014-04-10|2014|
+-----------+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------------+---------------+--------------------+---

In [60]:
data_upsert = [ {"marketplace":'US',"customer_id":'38602100', "review_id":'R315TR7JY5XODE',"product_id":'B00CHSWG6O',"product_parent":'336289302',"product_title" :'Amazon eGift Card', "star_rating":'5', "helpful_votes":'2',"total_votes":'0',"vine":'N',"verified_purchase":'Y',"review_headline":'GREAT',"review_body":'GOOD PRODUCT',"review_date":'2014-04-11',"year":'2014'},
{"marketplace":'US',"customer_id":'38602103', "review_id":'R315TR7JY5XOA1',"product_id":"B007V6EVY2","product_parent":'910961751',"product_title" :'Amazon eGift Card', "star_rating":'5', "helpful_votes":'2',"total_votes":'0',"vine":'N',"verified_purchase":'Y',"review_headline":'AWESOME',"review_body":'GREAT PRODUCT',"review_date":'2014-04-11',"year":'2014'}
]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [61]:
df_data_upsert = spark.createDataFrame(data_upsert)
df_data_upsert.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-------------+-----------+----------+--------------+-----------------+-------------+-----------+---------------+--------------+-----------+-----------+-----------------+----+----+
|customer_id|helpful_votes|marketplace|product_id|product_parent|    product_title|  review_body|review_date|review_headline|     review_id|star_rating|total_votes|verified_purchase|vine|year|
+-----------+-------------+-----------+----------+--------------+-----------------+-------------+-----------+---------------+--------------+-----------+-----------+-----------------+----+----+
|   38602100|            2|         US|B00CHSWG6O|     336289302|Amazon eGift Card| GOOD PRODUCT| 2014-04-11|          GREAT|R315TR7JY5XODE|          5|          0|                Y|   N|2014|
|   38602103|            2|         US|B007V6EVY2|     910961751|Amazon eGift Card|GREAT PRODUCT| 2014-04-11|        AWESOME|R315TR7JY5XOA1|          5|          0|                Y|   N|2014|
+-----------+-------------+--------

In [62]:
(deltaTable
.alias('t')
.merge(df_data_upsert.alias('u'), 't.review_id = u.review_id')
.whenMatchedUpdateAll()
.whenNotMatchedInsertAll()
.execute())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
spark.sql("select * from aws_product_review where review_id in ('R315TR7JY5XODE', 'R315TR7JY5XOA1')").show() # show sau khi khi upsert

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-----------+--------------+----------+--------------+-----------------+-----------+-------------+-----------+----+-----------------+---------------+-------------+-----------+----+
|marketplace|customer_id|     review_id|product_id|product_parent|    product_title|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline|  review_body|review_date|year|
+-----------+-----------+--------------+----------+--------------+-----------------+-----------+-------------+-----------+----+-----------------+---------------+-------------+-----------+----+
|         US|   38602100|R315TR7JY5XODE|B00CHSWG6O|     336289302|Amazon eGift Card|          5|            2|          0|   N|                Y|          GREAT| GOOD PRODUCT| 2014-04-11|2014|
|         US|   38602103|R315TR7JY5XOA1|B007V6EVY2|     910961751|Amazon eGift Card|          5|            2|          0|   N|                Y|        AWESOME|GREAT PRODUCT| 2014-04-11|2014|
+-----------+-----------+----------